In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
from joblib import Parallel, delayed
import os

import sys
sys.path.append(os.environ['GORDON_REPO_DIR'] + '/pipeline_scripts')
from utilities2014 import *

import matplotlib.pyplot as plt
%matplotlib inline

In [52]:
dm = DataManager(generate_hierarchy=False, stack='RS141', resol='x5', section=1)
dm._load_image()

mys, mxs = np.where(dm.mask)
mys = mys.astype(np.int16)
mxs = mxs.astype(np.int16)

n_theta = 12
theta_binwidth = 2*np.pi/n_theta

radius_list = [50,100,200]
n_radius = len(radius_list)
max_radius = radius_list[-1]


In [53]:
Gmax = dm.load_pipeline_result('Gmax', 'npy')
alpha = 1.5
Gmax = 1 - 1/(1+alpha*Gmax)

In [54]:
Gmax_viz = plt.cm.jet(Gmax/np.nanmax(Gmax));
# display(Gmax_viz)
# dm.save_pipeline_result(Gmax_viz, 'Gmax', 'jpg')

In [55]:
# def nonmaximum_suppression(Gmax):

dy, dx = np.gradient(Gmax)

grad_norm = np.sqrt(dx**2+dy**2)
grad_unit_vec_y = np.nan_to_num(dy/grad_norm)
grad_unit_vec_x = np.nan_to_num(-dx/grad_norm)

gy = grad_unit_vec_y[mys, mxs]
gx = grad_unit_vec_x[mys, mxs]

a = np.arange(-20, 20)

neighborhood_y = np.outer(gy, a)
neighborhood_x = np.outer(gx, a)

global_neighborhood_y = (mys[:,None] + neighborhood_y).astype(np.int)
global_neighborhood_x = (mxs[:,None] + neighborhood_x).astype(np.int)

b = time.time()
global_neighborhood_y[global_neighborhood_y < 0] = 0
global_neighborhood_y[global_neighborhood_y >= dm.image_height] = dm.image_height-1
global_neighborhood_x[global_neighborhood_x < 0] = 0
global_neighborhood_x[global_neighborhood_x >= dm.image_width] = dm.image_width-1
print time.time()-b

# 60s, slower than above
# b = time.time()
# global_neighborhood_y = np.minimum(np.maximum(global_neighborhood_y.astype(np.int), 0), height)
# global_neighborhood_x = np.minimum(np.maximum(global_neighborhood_x.astype(np.int), 0), width)
# print time.time()-b

global_neighborhood_values = Gmax[global_neighborhood_y, global_neighborhood_x]
global_neighborhood_maximum = global_neighborhood_values.max(axis=1)

values = Gmax[mys, mxs]

is_local_maximum = global_neighborhood_maximum == values

mys_local_max = mys[is_local_maximum]
mxs_local_max = mxs[is_local_maximum]

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/IPython/kernel/__main__.py:6: RuntimeWarning: invalid value encountered in divide
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/IPython/kernel/__main__.py:7: RuntimeWarning: invalid value encountered in divide


2.71691989899


In [56]:
G_nonmaxsup = np.zeros_like(Gmax)
G_nonmaxsup[mys_local_max, mxs_local_max] = Gmax[mys_local_max, mxs_local_max]
G_nonmaxsup[(grad_unit_vec_y==0)&(grad_unit_vec_x==0)] = 0

dm.save_pipeline_result(G_nonmaxsup, 'Gnonmaxsup', 'npy')

G_nonmaxsup_viz = plt.cm.jet(G_nonmaxsup/np.nanmax(G_nonmaxsup));
dm.save_pipeline_result(G_nonmaxsup_viz, 'Gnonmaxsup', 'jpg')

saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0001/RS141_x5_0001_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_Gnonmaxsup.npy
saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0001/RS141_x5_0001_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_Gnonmaxsup.jpg


/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/io/_io.py:159: UserWarning: /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0001/RS141_x5_0001_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_Gnonmaxsup.jpg is a low contrast image
  warnings.warn('%s is a low contrast image' % fname)


In [43]:
G_nonmaxsup_thresh_viz = dm.image_rgb.copy()

for y,x in zip(*np.where(G_nonmaxsup)):
    if Gmax[y,x] > .6:
        r = int(Gmax[y,x]*5)
        G_nonmaxsup_thresh_viz[y-r:y+r, x-r:x+r] = [Gmax[y,x], 0, 0]
        
dm.save_pipeline_result(G_nonmaxsup_thresh_viz, 'GnonmaxsupThresh', 'jpg')         
# display(G_nonmaxsup_thresh_viz)

saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0002/RS141_x5_0002_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_GnonmaxsupThresh.jpg


In [57]:
G_nonmaxsup_thresh_viz = np.zeros_like(dm.image)

for y,x in zip(*np.where(G_nonmaxsup)):
    if Gmax[y,x] > .6:
        r = int(Gmax[y,x]*5)
#         G_nonmaxsup_thresh_viz[y-r:y+r, x-r:x+r] = Gmax[y,x]
        G_nonmaxsup_thresh_viz[y-r:y+r, x-r:x+r] = 1

G_nonmaxsup_thresh_viz = 1. - G_nonmaxsup_thresh_viz
        
dm.save_pipeline_result(G_nonmaxsup_thresh_viz, 'GnonmaxsupThreshBW', 'jpg')
# display(G_nonmaxsup_thresh_viz)

saved /oasis/projects/nsf/csd395/yuncong/DavidData2014results/RS141/0001/RS141_x5_0001_gabor-blueNisslWide-segm-blueNisslRegular-vq-blueNissl_GnonmaxsupThreshBW.jpg


In [58]:
display(G_nonmaxsup_thresh_viz)

/oasis/projects/nsf/csd395/yuncong/Brain/notebooks/tmp.jpg